# RAG desde 0

![Modelo de Arquitectura](URL_de_la_imagen)

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [ ]:
# %pip install langchain langchain_community langchain_openai

In [ ]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    temperature=0.6,
)

Reconstruyamos el almacén de vectores del laboratorio pasado.

In [ ]:
import chromadb

chroma_client = chromadb.Client()

lat_collection = chroma_client.create_collection(name="lat_collection")

lat_collection.add(
    documents=[
        """Artículo 31. La alícuota del impuesto municipal a la actividad económica de
industria, comercio, servicios o de índole similar no podrá ser superior al tres por
ciento (3%) de los ingresos brutos obtenidos. El mínimo tributable anual para este
impuesto no podrá ser superior al equivalente en bolívares de doscientas cuarenta
veces el tipo de cambio de la moneda de mayor valor publicado por el Banco Central
de Venezuela.
Excepcionalmente, la alícuota del impuesto municipal a la actividad económica de
industria, comercio, servicios o de índole similar será de hasta seis coma cinco por
ciento (6,5%) de los ingresos brutos obtenidos, en los siguientes ramos:

1. Explotación de minas y canteras.
2. Servicios y construcción de industria petrolera.
3. Servicios de publicidad.
4. Venta al detal y/o mayor de bebidas alcohólicas.
5. Expendio de alimentos, bebidas y esparcimiento.
6. Bancos comerciales, instituciones financieras, seguros, administradoras y
actividades de índole similar.

7. Venta de joyas, relojes y piedras preciosas.
8. Fabricación de licores, tabacos, cigarrillos y derivados.""", 
        """Artículo 33. Las licencias o autorizaciones para el ejercicio de actividades
económicas, industriales, comerciales, de servicios y de índole similar sujetas a esta
Ley, tendrán una vigencia mínima de tres (3) años calendarios, contados a partir de
la fecha de su emisión por parte de la autoridad correspondiente, sin perjuicio del
pago de la tasa correspondiente por su mantenimiento anual.
La renovación de las licencias o autorizaciones a que se refiere este artículo
procederá de manera automática, bajo declaración jurada del solicitante sobre el
efectivo cumplimiento de todos los requisitos y trámites establecidos, previo pago
de los tributos que correspondan. Queda a salvo la facultad de las autoridades
competentes de revisar, en cualquier momento, la veracidad de la declaración
realizada por la persona solicitante.""",
        """Impuesto sobre instrumentos crediticios
Artículo 40. El impuesto estadal por el otorgamiento de instrumentos crediticios a
favor de personas naturales o jurídicas por parte de los bancos y demás instituciones
financieras, cuyas sucursales o agencias se encuentren ubicadas en la jurisdicción
de cada estado y del Distrito Capital, no podrá exceder de un bolívar por cada mil
bolívares (1x1000).
A tales efectos, se entenderá por instrumentos crediticios, aquellos mediante los
cuales los bancos y demás instituciones financieras otorguen de manera directa 
cantidades dinerarias, en calidad de préstamos y bajo las condiciones por ellos
estipuladas con excepción de las tarjetas de crédito y líneas de crédito.
El impuesto establecido en este artículo se causará al momento de la emisión del
instrumento crediticio."""
    ],
    metadatas=[
        {"source": "https://assets.kpmg.com/content/dam/kpmg/ve/pdf/2023/08/Gaceta-Oficial-6.755.pdf"}, 
        {"source": "https://assets.kpmg.com/content/dam/kpmg/ve/pdf/2023/08/Gaceta-Oficial-6.755.pdf"}, 
        {"source": "https://assets.kpmg.com/content/dam/kpmg/ve/pdf/2023/08/Gaceta-Oficial-6.755.pdf"}, 
    ],
    ids=["lat-01", "lat-02", "lat-03"]
)


In [ ]:
# Función para recuperar contenido de la base de datos de vectores

def query_vector_database(question):
    results = lat_collection.query(
        query_texts=[question],
        n_results=1
    )
    results_text = results['documents'][0][0]
    return results_text

In [ ]:
# Probemos la función

results_text = query_vector_database("Cuál es la alícuota máxima del impuesto de actividades económicas en Venezuela?")
print(results_text)

In [ ]:
# Usemos un simple prompt para realizar la generación aumentada

def prompt_template(question, context):
    return f'Lee el siguiente texto y responde esta pregunta: {question}. \nContexto: {context}'
 
def execute_llm_prompt(prompt_input):
    prompt_response = llm.invoke(prompt_input)
    return prompt_response


In [ ]:
# Probemos la función con una pregunta complicada que provoque una halucinación

pregunta_dificil = "How many columns have the three temples got in total?"
pd_resultado = query_vector_database(pregunta_dificil)
pd_prompt = prompt_template(pregunta_dificil, pd_resultado)
pd_respuesta = execute_llm_prompt(pd_prompt)
print(pd_respuesta)

TIP: Pueden conseguir prompts y otros recursos en Langchain Hub (https://smith.langchain.com/hub)

In [ ]:
# Mejoremos el prompt
def prompt_template(question, text):
    return f'Usa las siguientes partes del contexto recuperado para responder la pregunta. Si no sabes la respuesta, simplemente di que no la sabes. Usa un máximo de tres oraciones y mantén la respuesta concisa. \nPregunta: {question}\nContexto: {text}'


In [ ]:
# Probemos la función con una pregunta complicada que provoque una halucinación

pregunta_dificil = "Cuál es el máximo de las alicuotas de actividades económicas y de impuesto al trabajo?"
pd_resultado = query_vector_database(pregunta_dificil)
pd_prompt = prompt_template(pregunta_dificil, pd_resultado)
pd_respuesta = execute_llm_prompt(pd_prompt)
print(pd_respuesta)

In [ ]:
# Implementemos el chatbot

def chatbot(question):
    results_text = query_vector_database(question)
    prompt_input = prompt_template(question, results_text)
    prompt_output = execute_llm_prompt(prompt_input)
    return prompt_output

In [ ]:
# Probemos el chatbot

result = chatbot("Cuál es el máximo de las alicuotas de actividades económicas en el aforo de zapaterías?")
print(result)